<a href="https://colab.research.google.com/github/arina080803/DeepXML-ASTEC/blob/Colab-data-preparing/Custom_data_for_DeepXML%2BASTEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import json
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import FastText

In [9]:
df = pd.read_excel('/content/Коды для простой модели.xlsx', header=None)
df = df.dropna()
df

,0,1
0,Код,Описание
1,7007212009,ЛОБОВОЕ АВТОМОБИЛЬНОЕ СТЕКЛО УПРОЧНЕННОЕ ЗАКА...
2,7007212009,ЛОБОВОЕ АВТОМОБИЛЬНОЕ СТЕКЛО УПРОЧНЕННОЕ ЗАКА...
3,7007212009,ЛОБОВОЕ АВТОМОБИЛЬНОЕ СТЕКЛО УПРОЧНЕННОЕ ЗАКА...
4,8418102001,КОМБИНИРОВАННЫЕ ХОЛОДИЛЬНИКИ-МОРОЗИЛЬНИКИ С Р...
...,...,...
489191,8544429009,"КАБЕЛИ ЭЛЕКТРИЧЕСКИЕ ИЗОЛИРОВАННЫЕ, НА НАПРЯЖ..."
489192,8538909200,"ЧАСТИ АППАРАТУРЫ ДЛЯ УПРАВЛЕНИЯ, НА НАПРЯЖЕНИ..."
489193,805102000,"АПЕЛЬСИНЫ СВЕЖИЕ, БОТАНИЧЕСКОЕ НАЗВАНИЕ ""CITR..."
489194,2924190009,"СТЕАРАМИД ""FINAWAX-S (MICROBEADS)"", ДЛЯ ПРИМЕ..."


In [7]:
# Переименование столбцов для удобства
df.columns = ['code', 'description']

df["description"] = df["description"].astype(str)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Использование TF-IDF Vectorizer для преобразования текстов в векторные представления
vectorizer = TfidfVectorizer(max_features=300)
X_train = vectorizer.fit_transform(train_df['description']).toarray()
X_test = vectorizer.transform(test_df['description']).toarray()

# Функция для преобразования вектора в формат признаков
def vector_to_features(vector):
    return ' '.join(f'{i}:{value}' for i, value in enumerate(vector) if value != 0)

# Сохранение текстовых файлов для описаний (features) и кодов (labels)
with open('trn_X_Xf.txt', 'w') as f_train_features, \
     open('trn_X_Y.txt', 'w') as f_train_labels, \
     open('tst_X_Xf.txt', 'w') as f_test_features, \
     open('tst_X_Y.txt', 'w') as f_test_labels:

    for i, (features, label) in enumerate(zip(X_train, train_df['code'])):
        f_train_features.write(f"{i} {vector_to_features(features)}\n")
        f_train_labels.write(f"{i} {label}\n")

    for i, (features, label) in enumerate(zip(X_test, test_df['code'])):
        f_test_features.write(f"{i} {vector_to_features(features)}\n")
        f_test_labels.write(f"{i} {label}\n")


# Подготовка surrogate_mapping.txt
num_rows = len(df)
unique_codes = df['code'].unique()
with open('surrogate_mapping.txt', 'w') as f:
    for i in range(num_rows):
        f.write(f"{i}\n")

# Подготовка valid_labels.txt
with open('valid_labels.txt', 'w') as f:
    for code in unique_codes:
        f.write(f"{code}\n")

# Подготовка data_stats.json
data_stats = {
    'n_labels': len(unique_codes),
    'n_trn': len(train_df),
    'n_tst': len(test_df),
    'average_labels_per_doc': train_df['code'].value_counts().mean()
}
with open('data_stats.json', 'w') as f:
    json.dump(data_stats, f)


# fasttextB_embeddings_300d.npy

Подготовка файла fasttextB_embeddings_300d.npy
Этот файл обычно создается с помощью внешнего инструмента для создания эмбеддингов.
Здесь предполагается, что файл с эмбеддингами уже существует.
Если нет, вы можете использовать gensim для создания таких эмбеддингов.

In [11]:
df.columns = ['code', 'description']

df["description"] = df["description"].astype(str)

# Подготовка текстовых данных для обучения модели FastText
descriptions = df['description'].tolist()
sentences = [desc.split() for desc in descriptions]

# Обучение модели FastText
model = FastText(sentences, vector_size=300, window=5, min_count=1, workers=4, sg=1)

# Создание эмбеддингов для всех уникальных слов в описаниях
vocabulary = model.wv.index_to_key
embeddings = np.array([model.wv[word] for word in vocabulary])

# Сохранение эмбеддингов в файл
np.save('fasttextB_embeddings_300d.npy', embeddings)

print("Эмбеддинги сохранены в файл fasttextB_embeddings_300d.npy")

Эмбеддинги сохранены в файл fasttextB_embeddings_300d.npy


# data_stats.json

In [12]:
import json

# Количество строк в датасете
num_rows = len(df)

# Количество уникальных кодов
unique_codes = df['code'].nunique()

# Создание значений для data_stats.json
data_stats = {
    "threshold": 1024,
    "method": 1,
    "surrogate": f"{num_rows},{1024},{1024}",
    "extreme": f"{num_rows},{unique_codes},{unique_codes - 207}"
}

# Сохранение данных в файл data_stats.json
with open('data_stats.json', 'w') as f:
    json.dump(data_stats, f, indent=4)

print("Файл data_stats.json создан")


Файл data_stats.json создан


# Surrogate_mapping.txt

In [15]:
# Подготовка surrogate_mapping.txt
num_rows = len(df)
unique_codes = df['code'].unique()
with open('surrogate_mapping.txt', 'w') as f:
    for code in df['code']:
        f.write(f"{code}\n")

# valid_lables.txt

In [19]:
# Подготовка valid_labels.txt
unique_codes = df['code'].unique()
num_unique_codes = len(unique_codes)

with open('valid_labels.txt', 'w') as f:
    for i in range(num_unique_codes):
        f.write(f"{i}\n")